In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from hyperopt.pyll import scope
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np, xgboost as xgb
from xgboost import  XGBRegressor
from sklearn.model_selection import KFold, train_test_split



In [ ]:
# Set seed
SEED = 42
np.random.seed(SEED)

# Load dữ liệu
data_path = 'C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv'
df = pd.read_csv(data_path)

# Chọn đặc trưng và mục tiêu
features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
target = 'Gas Used'

X = df[features].astype(float)
y = df[target].astype(float)

# Tách train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

# Chuẩn hóa
scaler_X = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler_X.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler_X.transform(X_test), columns=X.columns)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()


kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

# Đánh giá
mse_scores, mae_scores, rmse_scores, r2_scores = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled), start=1):
    X_tr = X_train_scaled.iloc[train_idx]
    y_tr = y_train_scaled[train_idx]
    X_val = X_train_scaled.iloc[val_idx]
    y_val = y_train_scaled[val_idx]

    # XGBoost (dùng CPU 'hist' cho tương thích rộng; nếu có CUDA, đổi thành 'gpu_hist')
    model = XGBRegressor(
        objective='reg:squarederror',
        eval_metric='rmse',      # đặt trong constructor
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        subsample=1.0,
        colsample_bytree=1.0,
        reg_alpha=0.0,
        reg_lambda=1.0,
        tree_method='hist',      # <-- dùng 'gpu_hist' nếu có GPU CUDA
        n_jobs=-1,
        random_state=SEED
    )

    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)

    mse  = mean_squared_error(y_val, y_pred)
    mae  = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_val, y_pred)

    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

    print(f"Fold {fold} - MSE: {mse:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.6f}")

# Tổng kết sau tất cả folds
print("\n=== Kết quả trung bình (scaled) ===")
print(f"Avg MSE : {np.mean(mse_scores):.6f}")
print(f"Avg MAE : {np.mean(mae_scores):.6f}")
print(f"Avg RMSE: {np.mean(rmse_scores):.6f}")
print(f"Avg R²  : {np.mean(r2_scores):.6f}")

TypeError: fit() got an unexpected keyword argument 'callbacks'